In [ ]:
# click하고 들어가는 방식은 selenium 사용
# find_element_by_css_selector 부분만 soup으로 대체 가능


In [52]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm_notebook

In [33]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [34]:
driver.get('https://www.gmarket.co.kr/')

In [35]:
driver.maximize_window()    #창이 최소화되면 버튼이 없어지는 경우가 있기 때문에 최대화 해주기 

In [36]:
# elements는 list이기 때문에 click 대상을 인덱스로 정해주어야 됨.
best_btn = driver.find_element_by_css_selector('#desktop_layout-header > div > div > div.box__navigation > div.box__service-all > ul > li:nth-child(1) > a')
best_btn.click()

In [ ]:
category_list = []
name_list = []
price_list = []

# 1. xpath 포멧팅 / 2. 전체 data 불러와서 인덱싱
for i in tqdm_notebook(range(200)):
    images = driver.find_elements_by_css_selector('img.lazy')   # refresh 될 때마다 images를 인식할 수 있게 for문 안에 넣어 줘야함
    images[i].click()
    driver.implicitly_wait(5)   # 5초 이내에 로딩이 되면 진행하고 5초 이후에 로딩이 안되더라도 진행
    category = driver.find_element_by_css_selector('body > div.location-navi > ul > li.on > a') # 하위 tag가 겹치는지 확인하고 중복되면 상위 tag 지워도 됨
    name = driver.find_element_by_css_selector('h1.itemtit')
    price = driver.find_element_by_css_selector('strong.price_real')
    category_list.append(category.text)
    name_list.append(name.text)
    price_list.append(price.text)
    driver.back()
    time.sleep(1)

In [76]:
# 중간에 끊겼을 때 해당 지점에서 재시작하는 방법은 없음ㅠㅠ
# for문 range로 조절한 후에 나중에 list 합치는 방법 사용, 중간에 끊겨도 지나온 data는 저장되어 있음
category_list1 = []
name_list1 = []
price_list1 = []

for i in tqdm_notebook(range(102,200)):
    images = driver.find_elements_by_css_selector('img.lazy') 
    images[i].click()
    driver.implicitly_wait(5)
    try:
        category = driver.find_element_by_css_selector('li.on > a')
        name = driver.find_element_by_css_selector('h1.itemtit')
        price = driver.find_element_by_css_selector('strong.price_real')
        category_list1.append(category.text)
        name_list1.append(name.text)
        price_list1.append(price.text)
        driver.back()
        time.sleep(1)
    except:
        category = driver.find_element_by_xpath('/html/body/div[3]/ul/li[3]/a')
        name = driver.find_element_by_css_selector('h1.itemtit')
        price = driver.find_element_by_css_selector('strong.price_real')
        category_list1.append(category.text)
        name_list1.append(name.text)
        price_list1.append(price.text)
        driver.back()
        time.sleep(1)

C:\Users\user\AppData\Local\Temp\ipykernel_3328\2991869967.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(102,200)):


  0%|          | 0/98 [00:00<?, ?it/s]

In [81]:
# data 합치기
category_list_new = category_list + category_list1
name_list_new = name_list + name_list1
price_list_new = price_list + price_list1

In [74]:
#container > div.location-navi > ul > li.on > a
#body > div.location-navi > ul > li.on > a
#하위 tag가 중복이 되는지 확인 후 중복 된다면 상위 tag 지워도 됨
category = driver.find_element_by_css_selector('li.on > a').text
category

'강원'

In [88]:
market_data = {'카테고리':category_list_new,'상품명':name_list_new,'가격':price_list_new}
market = pd.DataFrame(market_data)
market

,카테고리,상품명,가격
0,메가박스,(스마일클럽)일반예매권2D-주중/주말,"8,500원"
1,생수,스파클생수 2L 30병,"15,900원"
2,새우,생물 새우 1kg 35-45미/14900원(중복할인20%+스마일12% 쿠폰가)/국내산,"21,900원"
3,브이넥니트,너의지금 20%쿠폰 신상니트 가디건/롱/세트/가을신상/크롭/박스/루즈핏/긴팔,"8,900원"
4,전집/세트,(타임주니어) Around the World / Magic of mirror 선택구매,"42,300원"
...,...,...,...
195,기타게임/퍼즐,인기게임218종모음 루미큐브 할리갈리 러시아워 카탄,"27,000원"
196,세탁세제/비누,비트 딥클린파워 액체세제 4L 용기 x 3개 (BOX) +증정비트 액체,"21,900원"
197,변기세정제,청개구리 자동 변기 클리너 세정제 6개 (12개월분),"19,800원"
198,가정용게임기,PS5 디스크 에디션 PlayStation 5 플스5 본체 1118,"688,000원"


In [ ]:
# 5초 이내에 로딩이 되면 진행하고 5초 이후에 로딩이 안되더라도 진행
driver.implicitly_wait(5)

#time.sleep은 로딩이 되든 안되든 5초를 기다리고 진행
time.sleep(5)